# 7. 卡尔曼滤波器的数学原理

In [2]:
%matplotlib inline

In [3]:
#format the book
import book_format
book_format.set_style()

如果你已经读到这里，我希望你觉得卡尔曼滤波器的可怕名声有些不太应该。当然，我有忽略了一些方程，但我希望你对实现过程还是比较理解的。其基本概念相当简单——取两个测量值，或者一次测量和一次预测，然后选择输出值介于两者之间。如果你更相信测量值，你的估计会更接近测量值；如果你更相信预测更准确，你的估计会更接近它。这不是什么高深的科学（开个小玩笑——正是这种数学让阿波罗号登月并安全返回了地球！）。

老实说，我一直精心选择问题。对于任意问题，设计卡尔曼滤波器的矩阵可能会非常困难。不过，我并没有搞得太复杂。像牛顿运动方程这样的方程在卡尔曼滤波器的应用中可以轻松计算，而且它们构成了我们想要解决的问题的大部分。

我用代码和推理阐述了概念，而不是数学。但有些主题需要比我目前使用的数学更多。这一章介绍了你在本书后续内容中会用到的数学知识。

# 动态系统建模

动态系统是指其状态（位置、温度等）随时间演变的物理系统。微积分是研究变化值的数学，因此我们使用微分方程来建模动态系统。有些系统无法用微分方程建模，但在本书中我们不会遇到这些系统。

对动态系统建模通常是几门大学课程的主题。在某种程度上，几个学期的普通微分方程和偏微分方程，再加上一门控制系统理论的研究生课程，是无可替代的。如果你是一个业余爱好者，或者在工作中尝试解决一个非常具体的滤波问题，你可能没有时间和/或倾向去花费一年或更长时间来接受这种教育。

幸运的是，我可以介绍足够的理论知识，让我们能够为许多不同的卡尔曼滤波器创建系统方程。我的目标是让你能够阅读一篇论文，并理解得足够透彻，以便实现这些算法。背景数学很深奥，但在实践中，我们最终使用了一些简单的技术。

这是本书中纯数学内容最长的一部分。你需要掌握本节中的所有内容才能理解扩展卡尔曼滤波器（EKF），这是最常见的非线性滤波器。我确实介绍了一些不需要太多数学知识的更现代的滤波器。你可以选择现在快速浏览一遍，如果决定学习EKF再回头来复习。

我们需要首先了解卡尔曼滤波器使用的基本方程和假设。我们试图对现实世界的现象进行建模，那么我们需要考虑什么？

每个物理系统都有一个过程。例如，以特定速度行驶的汽车在固定的时间内行驶一定距离，其速度随加速度变化。我们用高中学过的著名的牛顿方程描述了这种行为。

$$
\begin{aligned}
v&=at\\
x &= \frac{1}{2}at^2 + v_0t + x_0
\end{aligned}
$$

一旦我们学了微积分，就能以这种形式看到它们：

$$ \mathbf v = \frac{d \mathbf x}{d t}, 
\quad \mathbf a = \frac{d \mathbf v}{d t} = \frac{d^2 \mathbf x}{d t^2}
$$

典型的汽车跟踪问题会让你计算在恒定速度或加速度下行驶的距离，就像我们在之前的章节中所做的那样。但是，当然我们知道实际情况并不仅限于此。没有汽车会在完美的道路上行驶。路上会有颠簸、风阻，还有山丘会使速度上下波动。

排除最琐碎的问题之外完美地对系统进行建模是不可能的。我们被迫进行简化。在任何时刻$t$，我们说真实状态（比如我们的汽车位置）是来自不完美模型的预测值再加上一些未知的*过程噪声*：

$$
x(t) = x_{pred}(t) + noise(t)
$$

这并不意味着$noise(t)$是我们可以通过分析推导出的函数。这只是一个事实陈述——我们总是可以将真实值描述为预测值加上过程噪声。“噪声”并不意味着随机事件。如果我们在大气中追踪一个抛出的球，并且我们的模型假设球在真空中，那么空气阻力的影响就是在这种情况下的过程噪声。

在接下来的部分，我们将学习将一组高阶微分方程转换为一组一阶微分方程的技巧。在转换后，没有噪声的系统模型是：

$$ \dot{\mathbf x} = \mathbf{Ax}$$

矩阵 $\mathbf A$ 被称为 *系统动态矩阵*，因为它描述了系统的动态。现在我们需要对噪声建模。我们将其称为 $\mathbf w$，并将其添加到方程中。

$$ \dot{\mathbf x} = \mathbf{Ax} + \mathbf w$$

$\mathbf w$ 这个命名可能让你觉得不太合适，但你很快就会看到，卡尔曼滤波器假设是*白噪声*。

最后，我们需要考虑系统中的任何输入。我们假设有一个输入 $\mathbf u$，并且存在一个线性模型来定义该输入如何改变系统。例如，踩汽车油门会加速汽车，重力会使球下落。这两者都是控制输入。我们需要一个矩阵 $\mathbf B$ 将 $u$ 转换为对系统的影响，并将其添加到我们的方程中：

$$ \dot{\mathbf x} = \mathbf{Ax} + \mathbf{Bu} + \mathbf{w}$$

这就是其中一个方程。卡尔曼博士尝试解决的方程之一，他找到了一个最优的估计器，如果我们假设了 $\mathbf w$ 的某些特性。

# 动态系统状态空间的表达

我们推导出了这个方程：

$$ \dot{\mathbf x} = \mathbf{Ax}+ \mathbf{Bu} + \mathbf{w}$$

然而，我们对 $\mathbf x$ 本身感兴趣，而不是它的导数。暂时忽略噪声，我们希望找到一个递归地以 $t_{k-1}$ 时刻的 $\mathbf x$ 表达 $t_k$ 时刻 $\mathbf x$ 值的方程：

$$\mathbf x(t_k) = \mathbf F(\Delta t)\mathbf x(t_{k-1}) + \mathbf B(t_k)\mathbf u (t_k)$$

惯例允许我们将 $\mathbf x(t_k)$ 写成 $\mathbf x_k$，这表示 $t$ 的第 $k$ 个值时 $\mathbf x$ 的值。

$$\mathbf x_k = \mathbf{Fx}_{k-1} + \mathbf B_k\mathbf u_k$$

矩阵 $\mathbf F$ 是我们熟悉的 *状态转移矩阵*，因为它能够在离散时间步之间转换状态的值。它与系统动态矩阵 $\mathbf A$ 非常相似。不同之处在于 $\mathbf A$ 模拟一组线性微分方程，是连续的。而 $\mathbf F$ 是离散的，表示一组线性方程（不是微分方程），它在离散的时间步长 $\Delta t$ 内将 $\mathbf x_{k-1}$ 转换为 $\mathbf x_k$。

找到这个矩阵通常是相当困难的。方程 $\dot x = v$ 是最简单的微分方程，我们可以轻松地对其进行积分：

$$ \int\limits_{x_{k-1}}^{x_k}  \mathrm{d}x = \int\limits_{0}^{\Delta t} v\, \mathrm{d}t $$
$$x_k-x_{k-1} = v \Delta t$$
$$x_k = v \Delta t + x_{k-1}$$

这个方程是*递归*的：我们根据其在 $k-1$ 时刻的值来计算 $k$ 时刻的 $x$ 值。这种递归形式使我们能够将系统（过程模型）表示为卡尔曼滤波器所需的形式：

$$\begin{aligned}
\mathbf x_k &= \mathbf{Fx}_{k-1}  \\
&= \begin{bmatrix} 1 & \Delta t \\ 0 & 1\end{bmatrix}
\begin{bmatrix}x_{k-1} \\ \dot x_{k-1}\end{bmatrix}
\end{aligned}$$

我们之所以能这样做，只是因为 $\dot x = v$ 是可能的最简单的微分方程。几乎所有其他物理系统中的微分方程都更复杂，无法采用这种方法。

*状态空间* 方法在阿波罗任务时期变得流行，这在很大程度上要归功于卡尔曼博士的工作。其思想很简单。用一组 $n$ 阶微分方程对系统进行建模。将它们转换为等价的一阶微分方程集合。将它们放入前面章节中使用的向量-矩阵形式：$\dot{\mathbf x} = \mathbf{Ax} + \mathbf{Bu}$。一旦达到这种形式，我们就会使用几种技术将这些线性微分方程转换为递归方程：

$$ \mathbf x_k = \mathbf{Fx}_{k-1} + \mathbf B_k\mathbf u_k$$

一些书籍将状态转移矩阵称为*基本矩阵*。许多书籍使用 $\mathbf \Phi$ 而不是 $\mathbf F$。在很大程度上基于控制理论的资料倾向于使用这些形式。

这些被称为*状态空间*方法，因为我们是用系统状态来表达微分方程的解。

# 将高阶微分方程转化为一阶微分方程

许多物理系统的模型需要带有控制输入 $u$ 的二阶或更高阶微分方程：

$$a_n \frac{d^ny}{dt^n} + a_{n-1} \frac{d^{n-1}y}{dt^{n-1}} +  \dots + a_2 \frac{d^2y}{dt^2} + a_1 \frac{dy}{dt} + a_0 = u$$

状态空间方法需要一阶方程。任何高阶方程组都可以通过为导数定义额外的变量，然后进行求解来化简为一阶方程。

我们来做一个例子。给定系统 $\ddot{x} - 6\dot x + 9x = u$，找到等价的一阶方程。我用点符号表示时间导数以便更清楚。

第一步是将最高阶的项隔离到方程的一侧。

$$\ddot{x} = 6\dot x - 9x + u$$

我们定义两个新变量:

$$\begin{aligned} x_1(t) &= x \\
x_2(t) &= \dot x
\end{aligned}$$

现在我们将这些代入原方程并求解。解得到一组关于这些新变量的一阶方程。通常为了方便起见，我们在符号表示中省略 $(t)$。

我们知道 $\dot x_1 = x_2$，以及 $\dot x_2 = \ddot{x}$。因此

$$\begin{aligned}
\dot x_2 &= \ddot{x} \\
         &= 6\dot x - 9x + u\\
         &= 6x_2-9x_1 + u
\end{aligned}$$

因此，我们得到的一阶方程组是：

$$\begin{aligned}\dot x_1 &= x_2 \\
\dot x_2 &= 6x_2-9x_1 + u\end{aligned}$$

练习一段时间你就能熟练掌握了。将最高阶项隔离出来，定义一个新变量及其导数，然后进行代换。

# 一阶微分方程的状态空间形式

从前一节定义的新变量代入：

$$\frac{dx_1}{dt} = x_2,\,  
\frac{dx_2}{dt} = x_3, \, ..., \, 
\frac{dx_{n-1}}{dt} = x_n$$

代入一阶方程得到：

$$\frac{dx_n}{dt} = \frac{1}{a_n}\sum\limits_{i=0}^{n-1}a_ix_{i+1} + \frac{1}{a_n}u
$$

使用向量-矩阵表示法，我们有：

$$\begin{bmatrix}\frac{dx_1}{dt} \\ \frac{dx_2}{dt} \\ \vdots \\ \frac{dx_n}{dt}\end{bmatrix} = 
\begin{bmatrix}\dot x_1 \\ \dot x_2 \\ \vdots \\ \dot x_n\end{bmatrix}=
\begin{bmatrix}0 & 1 & 0 &\cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
-\frac{a_0}{a_n} & -\frac{a_1}{a_n} & -\frac{a_2}{a_n} & \cdots & -\frac{a_{n-1}}{a_n}\end{bmatrix}
\begin{bmatrix}x_1 \\ x_2 \\ \vdots \\ x_n\end{bmatrix} + 
\begin{bmatrix}0 \\ 0 \\ \vdots \\ \frac{1}{a_n}\end{bmatrix}u$$

然后我们将其写作 $\dot{\mathbf x} = \mathbf{Ax} + \mathbf{B}u$。

# 寻找时不变系统的基本矩阵

我们用状态空间形式表示系统方程，其中

$$ \dot{\mathbf x} = \mathbf{Ax}$$

其中 $\mathbf A$ 是系统动态矩阵，我们想要找到*基本矩阵* $\mathbf F$，它用以下方程在时间间隔 $\Delta t$ 内传播状态 $\mathbf x$：

$$\begin{aligned}
\mathbf x(t_k) = \mathbf F(\Delta t)\mathbf x(t_{k-1})\end{aligned}$$

换句话说，$\mathbf A$ 是一组连续微分方程，而我们需要 $\mathbf F$ 是一组离散线性方程，用于计算 $\mathbf A$ 在离散时间步长内的变化。

通常会省略 $t_k$ 和 $(\Delta t)$，使用记号

$$\mathbf x_k = \mathbf {Fx}_{k-1}$$

总的来说，有三种常用的方法来为卡尔曼滤波器找到这个矩阵。最常用的技术是矩阵指数。线性时不变理论，也称为LTI系统理论，是第二种技术。最后，还有数值技术。你可能知道其他方法，但在卡尔曼滤波器的文献和实践中，最常遇到的就是这三种。

# 矩阵指数

求解方程 $\frac{dx}{dt} = kx$ 可以通过以下步骤找到：

$$\begin{gathered}\frac{dx}{dt} = kx \\
\frac{dx}{x} = k\, dt \\
\int \frac{1}{x}\, dx = \int k\, dt \\
\log x = kt + c \\
x = e^{kt+c} \\
x = e^ce^{kt} \\
x = c_0e^{kt}\end{gathered}$$

当 $t=0$ 时，$x=x_0$。将这些代入上述方程。

$$\begin{gathered}x_0 = c_0e^{k(0)} \\
x_0 = c_01 \\
x_0 = c_0 \\
x = x_0e^{kt}\end{gathered}$$

用类似的数学方法，一阶方程的解是：

$$\dot{\mathbf x} = \mathbf{Ax} ,\, \, \, \mathbf x(0) = \mathbf x_0$$

其中 $\mathbf A$ 是一个常数矩阵，是

$$\mathbf x = e^{\mathbf At}\mathbf x_0$$

将 $F = e^{\mathbf At}$ 代入，我们可以写成

$$\mathbf x_k = \mathbf F\mathbf x_{k-1}$$

这就是我们要找的形式！我们将寻找基本

矩阵的问题简化为寻找 $e^{\mathbf At}$ 的值。

$e^{\mathbf At}$ 被称为[矩阵指数](https://en.wikipedia.org/wiki/Matrix_exponential)。它可以用以下幂级数计算：

$$e^{\mathbf At} = \mathbf{I} + \mathbf{A}t  + \frac{(\mathbf{A}t)^2}{2!} + \frac{(\mathbf{A}t)^3}{3!} + ... $$

这个级数是通过对 $e^{\mathbf At}$ 进行泰勒级数展开得到的，我在这里不会详细介绍。

让我们利用这个来找到牛顿方程的解。假设速度为常数，用 $v$ 代替 $\dot x$，我们得到线性矩阵-向量形式

$$\begin{bmatrix}\dot x \\ \dot v\end{bmatrix} =\begin{bmatrix}0&1\\0&0\end{bmatrix} \begin{bmatrix}x \\ v\end{bmatrix}$$

这是一个一阶微分方程，所以我们可以设 $\mathbf{A}=\begin{bmatrix}0&1\\0&0\end{bmatrix}$，并解下面的方程。我将间隔 $\Delta t$ 替换为 $t$，以强调基本矩阵是离散的：

$$\mathbf F = e^{\mathbf A\Delta t} = \mathbf{I} + \mathbf A\Delta t  + \frac{(\mathbf A\Delta t)^2}{2!} + \frac{(\mathbf A\Delta t)^3}{3!} + ... $$

如果你进行乘法运算，你会发现 $\mathbf{A}^2=\begin{bmatrix}0&0\\0&0\end{bmatrix}$，这意味着所有更高幂次的 $\mathbf{A}$ 也都是 $\mathbf{0}$。因此，我们得到了一个精确的答案，而不需要无穷多个项：

$$
\begin{aligned}
\mathbf F &=\mathbf{I} + \mathbf A \Delta t + \mathbf{0} \\
&= \begin{bmatrix}1&0\\0&1\end{bmatrix} + \begin{bmatrix}0&1\\0&0\end{bmatrix}\Delta t\\
&= \begin{bmatrix}1&\Delta t\\0&1\end{bmatrix}
\end{aligned}$$

我们将这个代入 $\mathbf x_k= \mathbf{Fx}_{k-1}$ 得到

$$
\begin{aligned}
x_k &=\begin{bmatrix}1&\Delta t\\0&1\end{bmatrix}x_{k-1}
\end{aligned}$$

这个矩阵正是我们在**多元卡尔曼滤波器**章节中对恒定速度卡尔曼滤波器进行解析推导得到的矩阵。

SciPy 的 linalg 模块包括一个用于计算矩阵指数的例程 `expm()`。它并不使用泰勒级数方法，而是使用[佩德算法近似](https://en.wikipedia.org/wiki/Pad%C3%A9_approximant)。有许多（至少19种）计算矩阵指数的方法，所有这些方法都存在数值困难[1]。你应该注意到这些问题，特别是当 $\mathbf A$ 很大时。如果你搜索“佩德算法矩阵指数”，你会找到许多致力于解决这个问题的论文。

在实践中，对于卡尔曼滤波器，我们通常只考虑泰勒级数的前两项，因此这可能不会成为你关注的问题。但不要假设我对这个问题的处理就是完整的，然后盲目尝试将这种技术用于其他问题，而不对这种技术的性能进行数值分析。有趣的是，解决 $e^{\mathbf At}$ 的一个受欢迎的方法是使用广义的常微分方程求解器。换句话说，他们做的与我们相反——将 $\mathbf A$ 转换成一组微分方程，然后使用数值技术求解这组方程！

这里是使用 `expm()` 解决 $e^{\mathbf At}$ 的一个例子。

In [4]:
import numpy as np
from scipy.linalg import expm

dt = 0.1
A = np.array([[0, 1], 
              [0, 0]])
expm(A*dt)

array([[1. , 0.1],
       [0. , 1. ]])

# 时不变系统

如果系统的行为取决于时间，我们可以说一个动态系统由一阶微分方程描述：

$$ g(t) = \dot x$$

然而，如果系统是*时不变*的，方程的形式则为：

$$ f(x) = \dot x$$

什么是*时不变*呢？如果你在时间 $t$ 输入信号 $x$，它将输出一些信号 $f(x)$。如果你在时间 $t + \Delta t$ 进行输入，输出信号将是相同的 $f(x)$，只是在时间上有所偏移。

一个反例是 $x(t) = \sin(t)$，其系统为 $f(x) = t\,  x(t) = t \sin(t)$。这不是时不变的；由于乘以 t，不同时间的值会有所不同。飞机也不是时不变的。如果你在稍后的时间对飞机进行控制输入，它的行为将会不同，因为它将消耗燃料，因此失去重量。较轻的重量会导致不同的行为。

我们可以通过对每一边进行积分来解这些方程。我之前展示了对时不变系统 $v = \dot x$ 进行积分。然而，对时不变方程 $\dot x = f(x)$ 进行积分并不那么直接。使用*分离变量*技术，我们除以 $f(x)$ 并将 $dt$ 项移到右边，以便对每一边进行积分：


$$\begin{gathered}
\frac{dx}{dt} = f(x) \\
\int^x_{x_0} \frac{1}{f(x)} dx = \int^t_{t_0} dt
\end{gathered}$$

如果我们令 $F(x) = \int \frac{1}{f(x)} dx$，我们会得到

$$F(x) - F(x_0) = t-t_0$$

然后我们解出 x，得到

$$\begin{gathered}
F(x) = t - t_0 + F(x_0) \\
x = F^{-1}[t-t_0 + F(x_0)]
\end{gathered}$$

换句话说，我们需要找到 $F$ 的反函数。这并不是简单的问题，在STEM教育中有很多课程专门致力于找到这个棘手的解析解。

然而，它们都是一些技巧，许多简单形式的 $f(x)$ 要么没有闭合形式的解，要么具有极大的困难。因此，实际工程师转向状态空间方法来寻找近似解。

矩阵指数的优势在于我们可以用它来处理任意一组*时不变*的微分方程。然而，即使方程并非时不变，我们通常也会使用这种技术。例如，随着飞机飞行，它会燃烧燃料并减轻重量。然而，在一秒钟内失重很小，因此系统在这个时间步长上几乎是线性的。只要时间步长很短，我们的答案仍然会相当准确。

# 例子：质量-弹簧-阻尼器模型

假设我们想要追踪弹簧上的一个质量，并连接到阻尼器，比如汽车的悬挂系统。其运动方程是 $m$ 为质量，$k$ 为弹簧常数，$c$ 为阻尼力，在某个输入 $u$ 下的运动方程为：

$$m\frac{d^2x}{dt^2} + c\frac{dx}{dt} +kx = u$$

为了符号的简便，我将其写成：

$$m\ddot x + c\dot x + kx = u$$

为了方便记号，我省略了 $(t)$。我可以通过设定 $x_1(t)=x(t)$ 来将其转化为一组一阶方程，并进行如下代换：

$$\begin{aligned}
x_1 &= x \\
x_2 &= \dot x_1 \\
\dot x_2 &= \ddot x_1 = \ddot x
\end{aligned}$$

这里常见的做法是省略了 $(t)$ 以方便记号。这给出了方程：

$$m\dot x_2 + c x_2 +kx_1 = u$$

解出 $\dot x_2$，我们得到一个一阶方程：

$$\dot x_2 = -\frac{c}{m}x_2 - \frac{k}{m}x_1 + \frac{1}{m}u$$

我们将其写成矩阵形式：

$$\begin{bmatrix} \dot x_1 \\ \dot x_2 \end{bmatrix} = 
\begin{bmatrix}0 & 1 \\ -\frac{k}{m} & -\frac{c}{m} \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} + 
\begin{bmatrix} 0 \\ \frac{1}{m} \end{bmatrix}u$$

现在我们使用矩阵指数来找到状态转移矩阵：

$$\Phi(t) = e^{\mathbf At} = \mathbf{I} + \mathbf At  + \frac{(\mathbf At)^2}{2!} + \frac{(\mathbf At)^3}{3!} + ... $$

前两项给出：

$$\mathbf F = \begin{bmatrix}1 & t \\ -\frac{k}{m} t & 1-\frac{c}{m} t \end{bmatrix}$$

这可能给出足够的精度，但也可能不够。你可以通过计算对应常数的 $\frac{(\mathbf At)^2}{2!}$ 并观察这个矩阵对结果的贡献来检验这一点。

# 线性时不变理论

[*线性时不变理论*](https://en.wikipedia.org/wiki/LTI_system_theory)，也称为LTI系统理论，为我们提供了一种使用逆拉普拉斯变换来找到 $\Phi$ 的方法。我在这本书中不会使用拉普拉斯变换。LTI系统理论告诉我们：

$$ \Phi(t) = \mathcal{L}^{-1}[(s\mathbf{I} - \mathbf{A})^{-1}]$$

我没有打算深入讨论这个问题，只是说一下拉普拉斯变换 $\mathcal{L}$ 将信号转换为一个不包括时间的空间 $s$，但找到上述方程的解并不简单。如果你感兴趣，可以参考维基百科上关于LTI系统理论的文章进行了解。我提到LTI是因为你会发现一些文献使用它来为复杂的问题设计卡尔曼滤波器的矩阵。

# 数值解

最后，还有数值技术来找到 $\mathbf F$。随着滤波器变得更大，找到解析解变得非常繁琐（尽管像SymPy这样的包使其更容易）。C. F. van Loan [2] 发展了一种技术来数值计算 $\Phi$ 和 $\mathbf Q$。给定连续模型

$$ \dot x = Ax + Gw$$

其中 $w$ 是单位白噪声，van Loan 的方法计算出 $\mathbf F_k$ 和 $\mathbf Q_k$。

我在 `FilterPy` 中实现了 van Loan 的方法。你可以这样使用：

```python
from filterpy.common import van_loan_discretization

A = np.array([[0., 1.], [-1., 0.]])
G = np.array([[0.], [2.]]) # white noise scaling
F, Q = van_loan_discretization(A, G, dt=0.1)
```

在 *数值积分的差分方程* 部分，我介绍了在卡尔曼滤波中非常常用的另一种替代方法。

# 过程噪声矩阵的设计

通常来说，设计 $\mathbf Q$ 矩阵是卡尔曼滤波器设计中最困难的部分之一。这是由于几个因素造成的。首先，数学需要对信号理论有良好的基础。其次，我们试图对我们知之甚少的某些事物中的噪声进行建模。想象一下试图为抛出的棒球建模过程噪声。我们可以将其建模为在空气中运动的一个球体，但这留下了许多未知因素——球的旋转和旋转衰减、带缝球的阻力系数、风力和空气密度的影响等等。我们为给定的过程模型开发了精确的数学解的方程，但由于过程模型不完整，$\mathbf Q$ 的结果也将不完整。这对卡尔曼滤波器的行为有很多影响。如果 $\mathbf Q$ 太小，滤波器将对其预测模型过于自信，并偏离实际解。如果 $\mathbf Q$ 太大，滤波器将受到测量中的噪声的不当影响，并表现不佳。在实践中，我们花费大量时间运行模拟并评估收集的数据，以尝试选择适当的 $\mathbf Q$ 值。但让我们首先看看数学原理。

假设一个运动系统——可以使用牛顿运动方程建模的某个系统。我们可以对这个过程作出几种不同的假设。

我们一直在使用一个过程模型：

$$ \dot{\mathbf x} = \mathbf{Ax} + \mathbf{Bu} + \mathbf{w}$$

其中 $\mathbf{w}$ 是过程噪声。运动系统是*连续*的——它们的输入和输出可以在任意时间点变化。然而，我们的卡尔曼滤波器是*离散*的（卡尔曼滤波器有连续形式，但我们在本书中不涉及）。我们以固定间隔对系统进行采样。因此，我们必须找到上述方程中噪声项的离散表示。这取决于我们对噪声行为的假设。我们将考虑两种不同的噪声模型。

# 连续白噪声模型

我们使用牛顿方程来建模运动系统。我们要么使用位置和速度，要么使用位置、速度和加速度作为系统的模型。我们可以进一步推广，比如可以模拟加加速度、跃度、猛度等。通常我们不这样做，因为超出实际系统动力学范围的项会降低估计的准确性。

假设我们需要对位置、速度和加速度进行建模。然后，我们可以假设每个离散时间步长内加速度是恒定的。当然，系统中存在过程噪声，因此实际上加速度并不是恒定的。由于未建模的外部力会随时间改变被跟踪对象的加速度。在这一部分，我们将假设加速度通过连续时间的零均值白噪声 $w(t)$ 发生变化。换句话说，我们假设速度的微小变化随时间平均为0（零均值）。

由于噪声在不断变化，我们需要进行积分，以获得我们选择的离散化间隔内的离散噪声。我们不会在这里证明，但离散噪声的方程是：

$$\mathbf Q = \int_0^{\Delta t} \mathbf F(t)\mathbf{Q_c}\mathbf F^\mathsf{T}(t) dt$$

其中 $\mathbf{Q_c}$ 是连续噪声。总体思路应该很清楚。$\mathbf F(t)\mathbf{Q_c}\mathbf F^\mathsf{T}(t)$ 是基于我们的过程模型 $\mathbf F(t)$ 在时刻 $t$ 的连续噪声的投影。我们想知道在离散时间间隔 $\Delta t$ 内系统添加了多少噪声，因此我们要对该表达式在区间 $[0, \Delta t]$ 上进行积分。

我们知道牛顿系统的基本矩阵是：

$$F = \begin{bmatrix}1 & \Delta t & {\Delta t}^2/2 \\ 0 & 1 & \Delta t\\ 0& 0& 1\end{bmatrix}$$

我们将连续噪声定义为:

$$\mathbf{Q_c} = \begin{bmatrix}0&0&0\\0&0&0\\0&0&1\end{bmatrix} \Phi_s$$

其中 $\Phi_s$ 是白噪声的谱密度。这个公式是可以推导出来的，但超出了本书的范围。详细信息请参考任何关于随机过程的标准文本。实际上，我们经常不知道噪声的谱密度，因此这变成了一个“工程”因素——一个我们通过实验调整的数字，直到我们的滤波器按预期运行为止。你可以看到 $\Phi_s$ 乘以的矩阵实际上将功率谱密度分配给了加速度项。这是有道理的；我们假设系统具有恒定的加速度，除了噪声引起的变化。噪声改变了加速度。

我们可以自己进行这些计算，但我更喜欢使用 SymPy 来解方程。

In [5]:
import sympy
from sympy import (init_printing, Matrix, MatMul, 
                   integrate, symbols)

init_printing(use_latex='mathjax')
dt, phi = symbols('\Delta{t} \Phi_s')
F_k = Matrix([[1, dt, dt**2/2],
              [0,  1,      dt],
              [0,  0,       1]])
Q_c = Matrix([[0, 0, 0],
              [0, 0, 0],
              [0, 0, 1]])*phi

Q = integrate(F_k * Q_c * F_k.T, (dt, 0, dt))

# factor phi out of the matrix to make it more readable
Q = Q / phi
MatMul(Q, phi)

⎡         5           4           3⎤      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥      
⎢    20          8           6     ⎥      
⎢                                  ⎥      
⎢         4           3           2⎥      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥⋅\Phiₛ
⎢    8           3           2     ⎥      
⎢                                  ⎥      
⎢         3           2            ⎥      
⎢\Delta{t}   \Delta{t}             ⎥      
⎢──────────  ──────────  \Delta{t} ⎥      
⎣    6           2                 ⎦      

为了完整起见，让我们计算零阶和一阶方程的公式。

In [6]:
F_k = Matrix([[1]])
Q_c = Matrix([[phi]])

print('0th order discrete process noise')
integrate(F_k*Q_c*F_k.T,(dt, 0, dt))

0th order discrete process noise


[\Delta{t}⋅\Phiₛ]

In [7]:
F_k = Matrix([[1, dt],
              [0, 1]])
Q_c = Matrix([[0, 0],
              [0, 1]]) * phi

Q = integrate(F_k * Q_c * F_k.T, (dt, 0, dt))

print('1st order discrete process noise')
# factor phi out of the matrix to make it more readable
Q = Q / phi
MatMul(Q, phi)

1st order discrete process noise


⎡         3           2⎤      
⎢\Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────⎥      
⎢    3           2     ⎥      
⎢                      ⎥⋅\Phiₛ
⎢         2            ⎥      
⎢\Delta{t}             ⎥      
⎢──────────  \Delta{t} ⎥      
⎣    2                 ⎦      